In [19]:
import torch
import stheno
import numpy as np
import matplotlib.pyplot as plt

from cnp.experiment import WorkingDirectory

import os
import pickle

root = '/scratches/cblgpu07/em626/kernelcnp/kernelcnp/experiments/synthetic'

In [20]:
# Directory where data are stored
data_name = 'eq-2-8-50-100-100-0'

# Path to data
data_dir = os.path.join(root, 'toy-data', data_name)
data_directory = WorkingDirectory(root=data_dir)

# Load data
file = open(data_directory.file('valid-data.pkl'), 'rb')
data = pickle.load(file)
file.close()

Root: /scratches/cblgpu07/em626/kernelcnp/kernelcnp/experiments/synthetic/toy-data/eq-2-8-50-100-100-0


In [37]:
# Generate data
def generate_dataset(data_type, num_context, num_total, noise=5e-2):
    
    # Set up covariance
    if data_type == 'eq':
        cov = stheno.EQ().stretch(1.)

    elif data_type == 'matern':
        cov = stheno.Matern52().stretch(1.)

    elif data_type == 'noisy-mixture-slow':
        cov = stheno.EQ().stretch(1.) + stheno.EQ().stretch(0.5)

    elif data_type == 'weakly-periodic-slow':
        cov = stheno.EQ().stretch(1.) * stheno.EQ().periodic(period=0.5)
        
    cov = cov + noise ** 2 * stheno.Delta()
    
    # Choose line to constrain along and choose inputs
    x0 = -2.
    x1 = 2.
    y0 = 4. * np.random.rand() - 2.
    y1 = 4. * np.random.rand() - 2.
    
    x = np.linspace(x0, x1, num_total)
    y = np.linspace(y0, y1, num_total)
    
    x = np.stack([x, y], axis=-1)
    
    # Sample data
    gp = stheno.GP(cov)
    sample = np.squeeze(gp(x).sample())
    
    idx = np.random.choice(np.arange(num_total), size=(num_context,))
    
    x_context = x[idx, :]
    y_context = sample[idx]
    x_target = np.delete(x, idx)
    y_target = np.delete(sample, idx)
    
    return x_context, x_target, y_context, y_target

In [41]:
data_type = 'eq'
num_context = 3
num_total = 500

x_context, x_target, y_context, y_target = generate_dataset(data_type=data_type,
                                                            num_context=num_context,
                                                            num_total=num_total)

In [39]:
# List models 

# Select input range

# For loop over models
#   Load model
#   Plot 1D marginals or samples
#   Save